Run inference and segmentation tasks locally 
==================================
This tutorial goes through steps to run inference and segmentation tasks with SEURON deployed locally by docker compose. **You should only use this notebook in JupyterLab hosted by a local SEURON deployment**. In order to run all the examples, you need a recent Linux system with **compatible GPU toolkit**. Make sure you follow the steps in the README file to create the local deployment.

The jupyterlab directory is mounted from a docker volume so the notebooks stored here are preserved until you delete the docker volume itself. A read-only copy of this tutorial is stored in the examples directory, in case you want to reverse changes made.

Load seuronbot extension
========================
In order to communicate with SEURON in a notebook, we need to load the seuronbot extension. **The seuronbot extension can only be loaded by one notebook, trying to load another instance of the extension will result in an error. If you want to use the extension in a different notebook, make sure you reset the kernel of the current notebook to unload the extension**

In [ ]:
%load_ext seuronbot_ext

Print a list of commands seuronbot supports
===========================================
In this tutorial we use jupyter frontend to demonstrate seuronbot commands, which are always initiated by the line/cell magic `seuronbot`. Depending on whether the command requires additional inputs, either the **line magic** `%seuronbot` or **cell magic** `%%seuronbot` should be used.

When using the **slack frontend** instead, one should replace `%seuronbot` and `%%seuronbot` by the bot's name. The name of the bot has the form of `seuron-worker-{DEPLOYMENT}`. For Google cloud deployment, `DEPLOYMENT` is the name you choose for your deployment, for local deployment, a randomly generated `DEPLOYMENT` is used and write to `.env.local` file. The bot will send a hello message in the notification channel you specified when setting up the deployment.

In [ ]:
%seuronbot help

Playground: a small cutout from FlyWire
=====================================
In order to demonstrate the pipeline, we use a small cutout from FAFB v14, which is the image stack FlyWire is based on. The bounding box of the cutout is [38000, 11500, 3530, 38512, 12012, 3730] at 16nm x 16nm x 40nm voxel size. In the cell below, The cutout are visualized using neuroglancer.

In [ ]:
from IPython.display import IFrame, HTML

ng_link = "https://cj-find-path-test-dot-neuroglancer-dot-seung-lab.appspot.com/#!%7B%22dimensions%22:%7B%22x%22:%5B1.6e-8%2C%22m%22%5D%2C%22y%22:%5B1.6e-8%2C%22m%22%5D%2C%22z%22:%5B4e-8%2C%22m%22%5D%7D%2C%22position%22:%5B38266.5%2C11729.5%2C3616.5%5D%2C%22crossSectionScale%22:0.9999999999999981%2C%22projectionOrientation%22:%5B-0.24463512003421783%2C0.09365575760602951%2C-0.11994557827711105%2C-0.9575987458229065%5D%2C%22projectionScale%22:2157.378810418306%2C%22layers%22:%5B%7B%22type%22:%22image%22%2C%22source%22:%22precomputed://https://bossdb-open-data.s3.amazonaws.com/flywire/fafbv14%22%2C%22tab%22:%22source%22%2C%22name%22:%22fafbv14%22%7D%2C%7B%22type%22:%22annotation%22%2C%22source%22:%7B%22url%22:%22local://annotations%22%2C%22transform%22:%7B%22outputDimensions%22:%7B%22x%22:%5B1.6e-8%2C%22m%22%5D%2C%22y%22:%5B1.6e-8%2C%22m%22%5D%2C%22z%22:%5B4e-8%2C%22m%22%5D%7D%7D%7D%2C%22tool%22:%22annotateBoundingBox%22%2C%22tab%22:%22annotations%22%2C%22annotations%22:%5B%7B%22pointA%22:%5B38000%2C11500%2C3530%5D%2C%22pointB%22:%5B38512%2C12012%2C3730%5D%2C%22type%22:%22axis_aligned_bounding_box%22%2C%22id%22:%2215fc1f9ae16810f0dd8aa4db2ac5eceed4ff7f56%22%7D%5D%2C%22name%22:%22annotation%22%7D%5D%2C%22selectedLayer%22:%7B%22layer%22:%22annotation%22%7D%2C%22layout%22:%22xy-3d%22%7D"
display(IFrame(src=ng_link, width=1200, height=600))

Run inference to create affinity map
====================================
To segment the cutout we picked, we first need to generate affinity map for it. The procedure is divided into two steps: First one submits the parameters describing the tasks, using the `update parameters` command. The bot performs sanity checks to prevent common mistakes. After the sanity check succeeds. One can launch the pipeline with `run pipeline` commands.

Submit inference parameters
--------------------------------------------------------
The command will send the python code in the current cell to SEURON. The cell content must be **self-contained**, no reference to variables or modules from other cells. The cell have to define a `submit_parameter` function requiring no arguments. The function should return a dictionary containing the inference parameters or a list of dictionaries for several tasks. The bot will execute this function in a separate environment and use the returned parameters for sanity checks.

SEURON uses [the zettaai fork of chunkflow](https://github.com/ZettaAI/chunkflow) for inference tasks, `ranlu/chunkflow:zettaai` pointing to the latest build from the zettaiai. The parameters for each run can be divided into roughly two types: parameters for IO and parameters for the inference model. The IO parameters contain information of the input images: the location, resolution and bounding box, the output will be stored in `OUTPUT_PATH`, or `{OUTPUT_PREFIX}{NAME}` when `OUTPUT_PATH` is not defined. 
The parameters for the inference model must be adjusted for each model, the parameters here are suitable for [the FlyWire model](https://github.com/seung-lab/DeepEM/releases/tag/0.0.6).

For local processes, we store the final and intermediate results to `/tmp`, which is automatically mounted from the host for all containers managed by SEURON. If you want to use some other volume, make sure you add them to the deployment file and have proper permission.

In [ ]:
%%seuronbot update parameters
def submit_inference_parameters():
    bbox = [38000, 11500, 3530, 38512, 12012, 3730]
    io_param = {
        "IMAGE_PATH": "https://bossdb-open-data.s3.amazonaws.com/flywire/fafbv14",
        "IMAGE_FILL_MISSING": True,
        "IMAGE_RESOLUTION": [16, 16, 40],
        "BBOX": bbox,
        "OUTPUT_PREFIX": "file:///tmp/scratch/ng/test_aff/",
    }
    aff_param = {
        "INPUT_PATCH_SIZE": [128,128,20],
        "OUTPUT_PATCH_SIZE": [96,96,16],
        "OUTPUT_CHANNELS": 3,
        "INPUT_PATCH_OVERLAP_RATIO": 0.5,
        "CHUNKFLOW_IMAGE": "ranlu/chunkflow:zettaai",
        "ONNX_MODEL_PATH": "https://github.com/seung-lab/DeepEM/releases/download/0.0.6/flywire_v0.2.onnx",
    }
    return [
                {
                    "NAME": "aff_local_test",
                    **io_param,
                    **aff_param,
                },
           ]

def submit_parameters():
    inf_param = submit_inference_parameters()
    return inf_param

Launch inference run
--------------------
Once the sanity check succeeded, we can start the inference using `%seuronbot run pipeline` command. The first message after the run is triggered is the cancel token to use if you want to cancel the run.

In [ ]:
%seuronbot run pipeline

Segment the affinity map
========================
After the affinity map is generated, we use it to create segmentation of the Flywire cutout. The procedure is similar to the inference tasks, first use `update parameter` command to submit the parameters describing the tasks, then use `run pipeline` to trigger the run.

Submit segmentation parameters
----------------------------------
For segmentation tasks SEURON uses [ABISS](https://github.com/seung-lab/abiss), `ranlu/abiss:main` is built from the main branch. The command creates both flat segmentations and inputs for [PyChunkedGraph](https://github.com/seung-lab/PyChunkedGraph). The segmentation requires affinity map as an input, make sure you update the path to the affinity map if you modified it in the previous step.

In [ ]:
%%seuronbot update parameters
from datetime import datetime

def submit_segmentation_parameters(aff_path):
    bbox = [38000, 11500, 3530, 38512, 12012, 3730]
    now = datetime.now().strftime("%Y%m%d%H%M%S")
    io_param = {
        "SCRATCH_PREFIX": "file:///tmp/scratch/",
        "IMAGE_PATH": "https://bossdb-open-data.s3.amazonaws.com/flywire/fafbv14",
        "AFF_PATH": aff_path,
        "BBOX": bbox,
        "NG_PREFIX": "file:///tmp/scratch/ng/",
    }
    seg_param = {
        "WS_HIGH_THRESHOLD": "0.99",
        "WS_LOW_THRESHOLD": "0.01",
        "WS_SIZE_THRESHOLD": "200",
        "AGG_THRESHOLD": "0.2",
        "WORKER_IMAGE": "ranlu/abiss:main",
        "SKIP_SKELETON": True,
    }
    return [
                {
                    "NAME": f"seg_fp32_{now}",
                    **io_param,
                    **seg_param,
                },
           ]

def submit_parameters():
    aff_path = "file:///tmp/scratch/ng/test_aff/aff_local_test"
    run_param = submit_segmentation_parameters(aff_path)
    return run_param

Launch segmentation run
-----------------------
Same as inference, we can start the segmentation using `%seuronbot run pipeline` command. And you will receive the cancel token at the starting of the run

In [ ]:
%seuronbot run pipeline

Combine runs
============
Instead of running single inference run or segmentation run, one can send parameters of multiple runs with a single `update parameters` command or even combine inference and segmentation. For example we can combine the previous two runs together

In [ ]:
%%seuronbot update parameters
from datetime import datetime

bbox = [38000, 11500, 3530, 38512, 12012, 3730]
def submit_segmentation_parameters(aff_path):
    now = datetime.now().strftime("%Y%m%d%H%M%S")
    io_param = {
        "SCRATCH_PREFIX": "file:///tmp/scratch/",
        "IMAGE_PATH": "https://storage.googleapis.com/neuroglancer-fafb-data/fafb_v14/fafb_v14_clahe",
        "AFF_PATH": aff_path,
        "BBOX": bbox,
        "NG_PREFIX": "file:///tmp/scratch/ng/",
    }
    seg_param = {
        "WS_HIGH_THRESHOLD": "0.99",
        "WS_LOW_THRESHOLD": "0.01",
        "WS_SIZE_THRESHOLD": "200",
        "AGG_THRESHOLD": "0.2",
        "WORKER_IMAGE": "ranlu/abiss:main",
        "SKIP_SKELETON": True,
    }
    return [
                {
                    "NAME": f"seg_fp32_{now}",
                    **io_param,
                    **seg_param,
                },
           ]


def submit_inference_parameters():
    now = datetime.now().strftime("%Y%m%d%H%M%S")
    io_param = {
        "IMAGE_PATH": "https://storage.googleapis.com/neuroglancer-fafb-data/fafb_v14/fafb_v14_clahe",
        "IMAGE_FILL_MISSING": True,
        "IMAGE_RESOLUTION": [16, 16, 40],
        "BBOX": bbox,
        "OUTPUT_PREFIX": "file:///tmp/scratch/ng/test_aff/",
    }
    aff_param = {
        "INPUT_PATCH_SIZE": [128,128,20],
        "OUTPUT_PATCH_SIZE": [96,96,16],
        "OUTPUT_CHANNELS": 3,
        "INPUT_PATCH_OVERLAP_RATIO": 0.5,
        "CHUNKFLOW_IMAGE": "ranlu/chunkflow:ort",
        "ONNX_MODEL_PATH": "https://github.com/seung-lab/DeepEM/releases/download/0.0.6/flywire_v0.2.onnx",
    }
    return [
                {
                    "NAME": f"aff_{now}",
                    **io_param,
                    **aff_param,
                },
           ]

def submit_parameters():
    inf_param = submit_inference_parameters()
    aff_path = inf_param[0]['OUTPUT_PREFIX']+inf_param[0]["NAME"]
    seg_param = submit_segmentation_parameters(aff_path)
    return inf_param+seg_param

In [ ]:
%seuronbot run pipeline